# Rigidity Results for c-Polyhedra

## John C. Bowers

### James Madison University Computer Science

Joint work with Philip L. Bowers (FSU) and Kevin Pratt (Stanford)

In [172]:
# Imports

# Algorithms:
from koebe.algorithms.incrementalConvexHull import incrConvexHull, orientationPointE3, randomConvexHullE3
from koebe.algorithms.hypPacker import *
from koebe.algorithms.sampling import surfaceSampling, boundarySampling
from koebe.algorithms.poissonDiskSampling import slowAmbientSurfaceSampling, slowAmbientBoundarySampling
from koebe.algorithms.cvt import weightedCVT, worldToImgPixelCoords
from koebe.algorithms.tutteEmbeddings import tutteEmbeddingE2

# Graphics:
from koebe.graphics.spherical2viewer import *
from koebe.graphics.euclidean2viewer import PoincareDiskViewer, makeStyle, E2Viewer

# Geometries:
from koebe.geometries.orientedProjective2 import DiskOP2
from koebe.geometries.euclidean3 import PointE3, VectorE3, SegmentE3
from koebe.geometries.euclidean2 import *
from koebe.geometries.commonOps import inner_product31

# Linear Algebra:
import numpy as np
from numpy.linalg import matrix_rank, norm
from scipy.linalg import null_space

# Image creation:
from PIL import Image, ImageDraw, ImageFilter

# Other:
import random, math

def shrinkDisk(disk, factor):
    v = VectorE3(-disk.a/disk.d, -disk.b/disk.d, -disk.c/disk.d)
    vclose = ((v.norm() - 1.0) * factor + 1.0) * v.normalize()
    return DiskS2(vclose.x, vclose.y, vclose.z, -1.0)

def conical_cap(disk):
    return PointE3(-disk.a/disk.d, -disk.b/disk.d, -disk.c/disk.d)
    
def moveOut(circleE2, factor = 2.0):
    return CircleE2(
        PointE2(circleE2.center.x * factor, circleE2.center.y * factor), 
        circleE2.radius * factor
    )

# Tangency Circle Packing

In [173]:
from koebe.graphics.euclidean2viewer import UnitScaleE2Sketch, makeStyle

poly = randomConvexHullE3(8) # Generate a random polyhedron with 16 vertices. 
poly.outerFace = poly.faces[0] # Arbitrarily select an outer face. 
tutteGraph = tutteEmbeddingE2(poly) # Compute the tutte embedding of the polyhedron. 

vertexStylesStroke = [
    makeStyle(stroke="#CD2828", strokeWeight = 2),
    makeStyle(stroke="#3227D0", strokeWeight = 2),
    makeStyle(stroke="#29A848", strokeWeight = 2),
    makeStyle(stroke="#FFEA7C", strokeWeight = 2),
    makeStyle(stroke="#552684", strokeWeight = 2),
    makeStyle(stroke="#249CB6", strokeWeight = 2),
    makeStyle(stroke="#FF66F1", strokeWeight = 2),
    makeStyle(stroke="#FD9C28", strokeWeight = 2)
]
vertexStylesFill = [
    makeStyle(fill="#CD2828", strokeWeight = 0.75),
    makeStyle(fill="#3227D0", strokeWeight = 0.75),
    makeStyle(fill="#29A848", strokeWeight = 0.75),
    makeStyle(fill="#FFEA7C", strokeWeight = 0.75),
    makeStyle(fill="#552684", strokeWeight = 0.75),
    makeStyle(fill="#249CB6", strokeWeight = 0.75),
    makeStyle(fill="#FF66F1", strokeWeight = 0.75),
    makeStyle(fill="#FD9C28", strokeWeight = 0.75)
]

# Show the Tutte embedding 
viewer = UnitScaleE2Sketch()
segments = [(SegmentE2(e.aDart.origin.data, e.aDart.twin.origin.data), 
             makeStyle(stroke="#000", strokeWeight=0.5)) 
            for e in tutteGraph.edges]
viewer.addAll(segments)
viewer.addAll([(tutteGraph.verts[i].data, vertexStylesFill[i]) for i in range(len(tutteGraph.verts))])
viewer.show()

<IPython.core.display.Javascript object>

E2Sketch(height=500, objects='[[{"type": "SegmentE2", "endpoints": [[1.0, 0.0], [-0.4999999999999998, 0.866025…

In [174]:
# Compute the packing
dists = [(v.data - PointE3.O).normSq() for v in tutteGraph.verts]
closestToOriginIdx = dists.index(min(dists))
packing, _ = maximalPacking(
    tutteGraph, 
    num_passes=1000, 
    centerDartIdx = tutteGraph.darts.index(tutteGraph.verts[closestToOriginIdx].aDart)
)

# Store each vertex's index
for i in range(len(packing.verts)):
    packing.verts[i].name = i


In [134]:
scaleFact = 15.0

# Run this to view the circle packing
viewer = S2Viewer(800, 800)

# Get the disks to draw
blueStyle = makeStyle(stroke = "#00f", strokeWeight = 0.5)
diskSet = [(DiskOP2.fromCircleE2(moveOut(v.data.toPoincareCircleE2(), scaleFact)).toDiskS2(), 
            vertexStylesStroke[v.name])
           for v in packing.verts]
viewer.addAll(diskSet)
    
redStyle = makeStyle(stroke = "#f00", strokeWeight = 0.5)

viewer.show()

<IPython.core.display.Javascript object>

S2Sketch(height=800, objects='[[{"type": "DiskS2", "disk": [-10.611214986811575, -15.406782613210106, 3.384180…

In [175]:
# Run this to view the circle packing
viewer = S2Viewer(800, 800)

# Get the disks to draw
blueStyle = makeStyle(stroke = "#00f", strokeWeight = 0.5)
diskSet = [(DiskOP2.fromCircleE2(moveOut(v.data.toPoincareCircleE2(), scaleFact)).toDiskS2(), 
            vertexStylesStroke[v.name])
           for v in packing.verts]

#diskSet = [(shrinkDisk(d), s) for d, s in diskSet]
diskSet = [(shrinkDisk(d, 1.5), s) for d, s in diskSet]

viewer.addAll(diskSet)
    
redStyle = makeStyle(stroke = "#f00", strokeWeight = 0.5)

viewer.show()

<IPython.core.display.Javascript object>

S2Sketch(height=800, objects='[[{"type": "DiskS2", "disk": [-0.9830361085238843, 1.435647118528393, -0.2727708…

In [136]:
# Run this to view the circle packing
viewer = S2Viewer(800, 800)

# Get the disks to draw
blueStyle = makeStyle(stroke = "#00f", strokeWeight = 0.5)
diskSet = [(DiskOP2.fromCircleE2(moveOut(v.data.toPoincareCircleE2(), scaleFact)).toDiskS2(), 
            vertexStylesStroke[v.name])
           for v in packing.verts]

#diskSet = [(shrinkDisk(d), s) for d, s in diskSet]
diskSet = [(shrinkDisk(d, 0.5), s) for d, s in diskSet]

viewer.addAll(diskSet)
    
redStyle = makeStyle(stroke = "#f00", strokeWeight = 0.5)

viewer.show()

<IPython.core.display.Javascript object>

S2Sketch(height=800, objects='[[{"type": "DiskS2", "disk": [-0.6997858872788862, -1.016042842831679, 0.2231791…

In [152]:
# Run this to view the circle packing
viewer = S2Viewer(800, 800)

# Get the disks to draw
blueStyle = makeStyle(stroke = "#00f", strokeWeight = 0.5)
diskSet = [(DiskOP2.fromCircleE2(moveOut(v.data.toPoincareCircleE2(), scaleFact)).toDiskS2(), 
            vertexStylesStroke[v.name])
           for v in packing.verts]

#diskSet = [(shrinkDisk(d), s) for d, s in diskSet]
diskSet = [(shrinkDisk(d, 0.5), s) for d, s in diskSet]

viewer.addAll(diskSet)
    
grayStyle = makeStyle(stroke = "#222", strokeWeight = 0.5)

orthos = [(CPlaneS2.throughThreeDiskS2(*[diskSet[v.name][0] for v in f.vertices()]), 
           grayStyle)
           for f in packing.faces]

viewer.addAll(orthos)
viewer.show()

<IPython.core.display.Javascript object>

S2Sketch(height=800, objects='[[{"type": "DiskS2", "disk": [-0.6997858872788862, -1.016042842831679, 0.2231791…

In [153]:
# Run this to view the circle packing
viewer = S2Viewer(800, 800)

# Get the disks to draw
blueStyle = makeStyle(stroke = "#00f", strokeWeight = 0.5)
diskSet = [(DiskOP2.fromCircleE2(moveOut(v.data.toPoincareCircleE2(), scaleFact)).toDiskS2(), 
            vertexStylesStroke[v.name])
           for v in packing.verts]
viewer.addAll(diskSet)

koebe_poly = packing.duplicate(
    vdata_transform = lambda c : conical_cap(DiskOP2.fromCircleE2(moveOut(c.toPoincareCircleE2(), scaleFact)).toDiskS2())
)

viewer.add(koebe_poly)
viewer.show()

<IPython.core.display.Javascript object>

S2Sketch(height=800, objects='[[{"type": "DiskS2", "disk": [-10.611214986811575, -15.406782613210106, 3.384180…

In [139]:
# Run this to view the circle packing
viewer = S2Viewer(800, 800)

# Get the disks to draw
blueStyle = makeStyle(stroke = "#00f", strokeWeight = 0.5)
diskSet = [(DiskOP2.fromCircleE2(moveOut(v.data.toPoincareCircleE2(), scaleFact)).toDiskS2(), 
            vertexStylesStroke[v.name])
           for v in packing.verts]

#diskSet = [(shrinkDisk(d), s) for d, s in diskSet]
diskSet = [(shrinkDisk(d, 0.5 + random.random()), s) for d, s in diskSet]

viewer.addAll(diskSet)
    
redStyle = makeStyle(stroke = "#f00", strokeWeight = 0.5)

viewer.show()

<IPython.core.display.Javascript object>

S2Sketch(height=800, objects='[[{"type": "DiskS2", "disk": [-0.8389294919104752, -1.2180701574456763, 0.267555…

In [154]:
# Run this to view the circle packing
viewer = S2Viewer(800, 800)

# Get the disks to draw
d2i = {}
for i in range(len(packing.verts)):
    d2i[packing.verts[i].data] = i
    
blueStyle = makeStyle(stroke = "#00f", strokeWeight = 0.5)
diskSet = [(DiskOP2.fromCircleE2(moveOut(v.data.toPoincareCircleE2(), scaleFact)).toDiskS2(), 
            vertexStylesStroke[v.name])
           for v in packing.verts]

#diskSet = [(shrinkDisk(d), s) for d, s in diskSet]
diskSet = [(shrinkDisk(d, 0.5 + random.random()), s) for d, s in diskSet]

viewer.addAll(diskSet)
    
koebe_poly = packing.duplicate(
    vdata_transform = lambda c : conical_cap(diskSet[d2i[c]][0])
)

viewer.add(koebe_poly)

grayStyle = makeStyle(stroke = "#222", strokeWeight = 0.5)

orthos = [(CPlaneS2.throughThreeDiskS2(*[diskSet[v.name][0] for v in f.vertices()]), 
           grayStyle)
           for f in packing.faces]

viewer.addAll(orthos)

viewer.show()

<IPython.core.display.Javascript object>

S2Sketch(height=800, objects='[[{"type": "DiskS2", "disk": [-0.8446275276317283, -1.2263433285941983, 0.269372…

In [155]:
# Run this to view the circle packing
viewer = S2Viewer(800, 800)

# Get the disks to draw
d2i = {}
for i in range(len(packing.verts)):
    d2i[packing.verts[i].data] = i
    
blueStyle = makeStyle(stroke = "#00f", strokeWeight = 0.5)
diskSet = [(DiskOP2.fromCircleE2(moveOut(v.data.toPoincareCircleE2(), scaleFact)).toDiskS2(), 
            vertexStylesStroke[v.name])
           for v in packing.verts]

#diskSet = [(shrinkDisk(d), s) for d, s in diskSet]
diskSet = [(shrinkDisk(d, 0.5 + random.random()), s) for d, s in diskSet]

viewer.addAll(diskSet)
    


grayStyle = makeStyle(stroke = "#222", strokeWeight = 0.5)

orthos = [(CPlaneS2.throughThreeDiskS2(*[diskSet[v.name][0] for v in f.vertices()]), 
           grayStyle)
           for f in packing.faces]

viewer.addAll(orthos)

viewer.show()

<IPython.core.display.Javascript object>

S2Sketch(height=800, objects='[[{"type": "DiskS2", "disk": [-0.9111078526356262, -1.3228683652336053, 0.290575…

In [171]:
disk1, disk2 = diskSet[1], diskSet[2]
viewer = S2Viewer(800, 800)

viewer.addAll([disk1, disk2])

for t in range(1, 100):
    dt = t / 100.0
    disk = DiskS2(
        disk1[0].a * (1 - dt) - disk2[0].a * dt, 
        disk1[0].b * (1 - dt) - disk2[0].b * dt, 
        disk1[0].c * (1 - dt) - disk2[0].c * dt, 
        disk1[0].d * (1 - dt) - disk2[0].d * dt
    )
    viewer.addAll([disk1, disk2, (disk, grayStyle)])
    #viewer.addAll([conical_cap(disk1[0]), conical_cap(disk2[0]), conical_cap(disk)])
    viewer.pushAnimFrame()


for t in range(1, 100):
    dt = 1 - t / 100.0
    disk = DiskS2(
        disk1[0].a * (1 - dt) - disk2[0].a * dt, 
        disk1[0].b * (1 - dt) - disk2[0].b * dt, 
        disk1[0].c * (1 - dt) - disk2[0].c * dt, 
        disk1[0].d * (1 - dt) - disk2[0].d * dt
    )
    viewer.addAll([disk1, disk2, (disk, grayStyle)])
    viewer.pushAnimFrame()

viewer.show()

<IPython.core.display.Javascript object>

S2Sketch(height=800, objects='[[{"type": "DiskS2", "disk": [-0.9755347296738575, 1.1300891646081912, 0.9088005…